In [3]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv


from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import AzureChatOpenAI  

dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')


client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    },
)

print(client.is_ready())


True


In [6]:
COLLECTION_NAME = "recipes"

embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large")

db = WeaviateVectorStore(
    client=client, 
    index_name=COLLECTION_NAME, 
    text_key="chunk", 
    embedding=embeddings
)

In [12]:
prompt_template = """Text: {context}

Question: {question}

Answer the question based on the text provided. If the text doesn't contain the answer, 
reply that the answer is not available."""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
prompt_template



"Text: {context}\n\nQuestion: {question}\n\nAnswer the question based on the text provided. If the text doesn't contain the answer, \nreply that the answer is not available."

In [9]:

# source_file = "netherlands-wikipedia-article-text.pdf"
# where_filter = {
#       "operator": "Equal",
#       "path": ["source"],
#       "valueText": source_file
#   }

# we want our retriever to filter the results
# retriever = db.as_retriever(search_kwargs={"where_filter": where_filter})

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")


retriever = db.as_retriever()

openai_client = AzureChatOpenAI(
    model_name="gpt-35-turbo", 
    deployment_name = "gpt-35-turbo",
    )

qa_chain = RetrievalQA.from_llm(
    openai_client, retriever=retriever, prompt=prompt
)


qa_chain.invoke("How to make chilaquiles?")



/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/.venv/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'query': 'How to make chilaquiles?',
 'result': "I don't know how to make chilaquiles based on the provided context."}